This notebook serves for demonstrating some capabilites of [yamplpyowl](https://github.com/cknoll/yamlpyowl). It originated from [issue #4](https://github.com/cknoll/yamlpyowl/issues/4).


In [1]:
fpath = "./ontology_data/issue4-demo.yml"
with open(fpath, "r") as txtfile:
    print(txtfile.read())

- iri: http://yet.undefined.org/yamlpyowl/issue4-demo.owl
- annotation: >

    Mininal example to demonstrate some features of yamlpyowl Python package.
    See https://github.com/cknoll/yamlpyowl/issues/4 for background information

- owl_class:
    Class1:
        SubClassOf: "owl:Thing"

- owl_data_property:
    functieCode:
        Domain: Class1
        Range: int

- owl_class:
    Overeenkomst1:
        # note: this makes this class a subclass of Class1 due to the domain of the property
        SubClassOf:
            Or:
                - functieCode:
                    value: 100
                - functieCode:
                    value: 200
- owl_class:
    Overeenkomst2:
        # note: this makes this class a subclass of Class1 due to the domain of the property
        EquivalentTo:
            Or:
                - functieCode:
                    value: 100
                - functieCode:
                    value: 110

- owl_individual:
      instance1:
        types:
    

In [2]:
import yamlpyowl as ypo
om = ypo.OntologyManager(fpath)

In [3]:
# note: this version might not yet be relesed yet but is available at least in the develop branch of the repo
ypo.__version__

'0.3.0'

In [4]:
list(om.onto.classes())

[<<class 'owlready2.entity.ThingClass'> 'Class1'>,
 <<class 'owlready2.entity.ThingClass'> 'Overeenkomst1'>,
 <<class 'owlready2.entity.ThingClass'> 'Overeenkomst2'>]

### Gather some taxonomic information before and after running the reasoner

In [5]:
list(om.onto.Overeenkomst1.instances())

[]

In [6]:
list(om.onto.Overeenkomst2.instances())

[]

In [7]:
list(om.onto.Class1.subclasses())

[]

In [8]:
om.sync_reasoner()

In [9]:
list(om.onto.Overeenkomst1.instances())

[]

In [10]:
# the instance is reclassified as expected
list(om.onto.Overeenkomst2.instances())

[<<<class 'owlready2.entity.ThingClass'> 'Overeenkomst2'> 'instance1'>]

In [11]:
# due to the domain of the property (`Class1`) the reasoner inferes that the
# two classes must be subclasses of that one.
list(om.onto.Class1.subclasses())

[<<class 'owlready2.entity.ThingClass'> 'Overeenkomst1'>,
 <<class 'owlready2.entity.ThingClass'> 'Overeenkomst2'>]

Below is the old content of the notebook


---

This is a proposed implementation in owlready2-based Python (used earlier to avoid misunderstanding the semantics):

In [1]:
import owlready2 as owl2
onto = owl2.get_ontology("http://test.org/onto.owl")

with onto:
    class Class1(owl2.Thing):
        """
        - owl_class:
            Class1:
                SubClassOf: "owl:Thing"
        """
        pass

    class functieCode(owl2.DataProperty):
        """
        - owl_data_property:
            functieCode:
                Domain: Class1
                Range: int 
        """
        
        domain = [Class1]
        range = [int]
        
        
    # this is based on https://owlready2.readthedocs.io/en/v0.35/restriction.html
    class Overeenkomst(owl2.Thing):
        """
        - owl_class:
            Overeenkomst:
                SubClassOf: (functieCode value 100) or (functieCode value 110)
        """

        # in owlready2 a SubClassOf statement would use `is_a = ...`
        # this is syntactically correct, but does not enable the reasoner to reclassify
        # an instance based on its data property value.
        # thus, here we choose `equivalent_to` instead of `is_a`
        equivalent_to = [functieCode.value(100) | functieCode.value(110)]

# onto.save("value_restriction_test.xml")

In [2]:
test_instance0 = Class1("t0", functieCode=[100])
test_instance1 = Class1("t1", functieCode=[110])
test_instance2 = Class1("t2", functieCode=[120])

In [3]:
# evaluate the uninferred instances (there should be none)
list(Overeenkomst.instances())

[]

In [4]:
# evaluate the uninferred subclasses (there should be none)
list(Class1.subclasses())

[]

In [5]:
# run the reasoner
owl2.sync_reasoner(infer_property_values=True, debug=False)

In [6]:
# evaluate the inferred instances (there should be two)
list(Class1.instances())

[onto.t0, onto.t1, onto.t2]

In [7]:
# evaluate the inferred subclasses (there should be one)
# because the domain of `functieCode` is Class1, every instance for which that property can be defined,
# must be an instance of Class1.
list(Class1.subclasses())

[onto.Overeenkomst]